In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [2]:
!nvidia-smi

Mon Aug  9 08:50:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Create non-linear data distribution

In [3]:
m = 500
nonlinear_distribution = np.random.normal(0, 1, [m, 2])

In [4]:
nonlinear_distribution.shape

(500, 2)

In [6]:
import plotly.graph_objects as go
# 독립변수입력) x1, x2 => 종속변수(정답) y 
y = nonlinear_distribution[:,0]**2 + nonlinear_distribution[:,1]**2
fig = go.Figure(data=[go.Scatter3d(x=nonlinear_distribution[:,0], y=nonlinear_distribution[:,1], z=y,
                                   mode='markers',
                                  marker=dict(  size=4,
                                                colorscale='blues',   # choose a colorscale
                                                opacity=0.8
                                            ))])

fig.update_layout(
    title='Data distribution', 
    autosize=True,
    width=500, 
    height=500,
    xaxis=dict(range=[0.,10.]),
    margin=dict(l=65, r=50, b=65, t=90),
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=1, y=1, z=1),
    scene=dict(
        xaxis= dict(range=[-5,5], nticks=20),
        yaxis= dict(range=[-5,5], nticks=20),
        zaxis= dict(range=[0,20]),
        xaxis_title='X1',
        yaxis_title='X2',
        zaxis_title='Y',
    ))

fig.show()

# 2. Build your regression model with Neural Network

In [ ]:
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential([layers.Dense(32, activation='relu', input_shape=[2]),
                            layers.Dense(16, activation='relu'),
                            layers.Dense(2, activation='relu'),
                            layers.Dense(1)
                            ])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])

# 3. Train your model

In [ ]:
model.fit(nonlinear_distribution, y, epochs=1000)

In [ ]:
model.summary()

# 4. Observe your new representation of dataset transformed by network

In [ ]:
#학습시킨 모델의 layers 속성에 소속된 레이어 객체들의 리스트가 있습니다.
model.layers

In [ ]:
# 입력값에 대한 hidden layer의 출력값들을 얻어냅니다.
last_hidden_output_model = tf.keras.Sequential()

#마지막 레이어(출력층)을 제외한 다른 레이어를 새로운 Sequential 모델에 포함십니다.
for layer in model.layers[:-1]:
    last_hidden_output_model.add(layer)

In [ ]:
#출력층을 제외한 은닉층들이 포함됐습니다.
last_hidden_output_model.summary()

In [ ]:
#기존 데이터를 은닉층들에 통과시켜 결과를 얻습니다.
#transformed는 기존 데이터에 대한 새로운 데이터표상(representation)입니다.
transformed = last_hidden_output_model.predict(nonlinear_distribution)

In [ ]:
# 새로운 데이터값의 분포를 그려봅시다.

fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.set_title('nonlinear distribution')
ax1.set_xlabel('hidden x1')
ax1.set_ylabel('hidden x2')
y = nonlinear_distribution[:, 0]**2 + nonlinear_distribution[:,1]**2
#y = norm/norm.max()

ax1.scatter(transformed[:,0], transformed[:,1], c=y, cmap='gray')
plt.show()

In [ ]:
import plotly.graph_objects as go
# 독립변수입력) x1, x2 => 종속변수(정답) y 
fig = go.Figure(data=[go.Scatter3d(x=transformed[:,0], y=transformed[:,1], z=y,
                                   mode='markers',
                                  marker=dict(  size=8,
                                                colorscale='blues',   # choose a colorscale
                                                opacity=0.8
                                            ))])
fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y (Answer)'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig.show()